<a href="https://colab.research.google.com/github/greyhound101/coco/blob/main/preprare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [3]:
import shutil
shutil.copy('/content/gdrive/MyDrive/coco/VOCtest_06-Nov-2007.tar','/content/')
shutil.copy('/content/gdrive/MyDrive/coco/VOCtrainval_06-Nov-2007.tar','/content/')
shutil.copy('/content/gdrive/MyDrive/coco/VOCtrainval_11-May-2012.tar','/content/')

FileNotFoundError: ignored

In [4]:
shutil.copy('/content/gdrive/MyDrive/coco/annotations_trainval2017.zip','/content/')
shutil.copy('/content/gdrive/MyDrive/coco/voc_label.py','/content/')

'/content/voc_label.py'

In [10]:
import zipfile
with zipfile.ZipFile('/content/annotations_trainval2017.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [11]:
import tarfile
paths=['/content/VOCtest_06-Nov-2007.tar',
       '/content/VOCtrainval_06-Nov-2007.tar',
       '/content/VOCtrainval_11-May-2012.tar']
for fname in paths:
  tar = tarfile.open(fname, "r:")
  tar.extractall()
  tar.close()

In [12]:
!python voc_label.py

In [13]:
import xml.etree.ElementTree as ET
import pickle
import os
from os import listdir, getcwd
from os.path import join
import argparse


# parser = argparse.ArgumentParser()
# parser.add_argument('--type', type=str, choices=['1c', 'all'], required=True)
# args = parser.parse_args()


sets=[('2012', 'train'), ('2012', 'val'), ('2007', 'train'), ('2007', 'val'), ('2007', 'test')]

classes = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert_annotation(year, image_id, class_name):
    in_file = open('VOCdevkit/VOC%s/Annotations/%s.xml'%(year, image_id))
    out_file = open('VOCdevkit/VOC%s/labels_1c/%s/%s.txt'%(year, class_name, image_id), 'w')
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls != class_name or int(difficult) == 1:
            continue
        # cls_id = classes.index(cls)
        cls_id = 0
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')


wd = getcwd()

if not os.path.exists('voclist'):
    os.mkdir('voclist')

for class_name in classes:
    for year, image_set in sets:
        image_ids = open('VOCdevkit/VOC%s/ImageSets/Main/%s_%s.txt'%(year, class_name, image_set)).read().strip().split()
        ids, flags = image_ids[::2], image_ids[1::2]
        image_ids = list(zip(ids, flags))

        # File to save the image path list
        list_file = open('voclist/%s_%s_%s.txt'%(year, class_name, image_set), 'w')

        # File to save the image labels
        label_dir = 'labels_1c/' + class_name
        if not os.path.exists('VOCdevkit/VOC%s/%s/'%(year, label_dir)):
            os.makedirs('VOCdevkit/VOC%s/%s/'%(year, label_dir))

        # Traverse all images
        for image_id, flag in image_ids:
            if int(flag) == -1:
                continue
            list_file.write('%s/VOCdevkit/VOC%s/JPEGImages/%s.jpg\n'%(wd, year, image_id))
            convert_annotation(year, image_id, class_name)
        list_file.close()

    files = [
        'voclist/2007_{}_train.txt'.format(class_name),
        'voclist/2007_{}_val.txt'.format(class_name),
        'voclist/2012_{}_*.txt'.format(class_name)
    ]
    files = ' '.join(files)
    cmd = 'cat ' + files + '> voclist/{}_train.txt'.format(class_name)
    os.system(cmd)

In [19]:
import argparse
import random
import os
import numpy as np
from os import path
droot='/content'

droot = droot.rstrip('/')
tgt_folder = path.join(droot, 'voclist')
src_folder = 'data/vocsplit'

print('===> Converting few-shot name lists.. ')
for name_list in sorted(os.listdir(src_folder)):
    print('  | On ' + name_list)
    # Read from src
    with open(path.join(src_folder, name_list), 'r') as f:
        names = f.readlines()
    
    # Replace data root
    names = [name.replace('/scratch/bykang/datasets', droot) 
             for name in names]
    
    with open(path.join(droot, 'voclist', name_list), 'w') as f:
        f.writelines(names)

print('===> Converting class to namelist dict file ...')
for fname in ['voc_traindict_full.txt',
              'voc_traindict_bbox_1shot.txt',
              'voc_traindict_bbox_2shot.txt',
              'voc_traindict_bbox_3shot.txt',
              'voc_traindict_bbox_5shot.txt',
              'voc_traindict_bbox_10shot.txt']: 
    full_name = path.join('data', fname)
    print('  | On ' + full_name)
    # Read lines
    with open(full_name, 'r') as f:
        lines = f.readlines()

    # Replace data root
    lines = [line.replace('/scratch/bykang/datasets', droot) 
             for line in lines]

    # Rewrite linea
    with open(full_name, 'w') as f:
        f.writelines(lines)

print('===> Finished!')

===> Converting few-shot name lists.. 


FileNotFoundError: ignored